# postgresql_connection

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
# Install the PostgreSQL adapter for Python
!pip install psycopg2-binary sentence-transformers

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 23.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Retrieve credentials from the .env file
db_params = {
    "host": os.getenv("PG_HOST"),
    "port": os.getenv("PG_PORT"),
    "database": os.getenv("PG_DB"),
    "user": os.getenv("PG_USER"),
    "password": os.getenv("PG_PASSWORD")
}

# Connect to PostgreSQL and create the pgvector table
conn = None
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("✓ Successfully connected to PostgreSQL!")

    # Enable the pgvector extension
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("✓ Vector extension enabled.")

    # Create a table with a vector column for embeddings (dimension 384)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS documents (
        id BIGSERIAL PRIMARY KEY,
        content TEXT,
        embedding vector(384)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    print("✓ Table 'documents' created successfully (or already exists).")

except psycopg2.Error as e:
    print(f"✗ Error connecting to PostgreSQL: {e}")

finally:
    if conn:
        cursor.close()
        conn.close()
        print("Connection closed.")


✓ Successfully connected to PostgreSQL!
✓ Vector extension enabled.
✓ Table 'documents' created successfully (or already exists).
Connection closed.


In [2]:
# Re-establish connection for a new operation
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Insert a test record
insert_query = "INSERT INTO documents (content) VALUES (%s) RETURNING id;"
cursor.execute(insert_query, ("Hello, World! This is my first PostgreSQL document.",))
# Get the ID of the newly inserted record
new_id = cursor.fetchone()[0]

conn.commit()
print(f"✓ Inserted document with id: {new_id}")

cursor.close()
conn.close()

✓ Inserted document with id: 1


In [3]:
# ### CLEANUP CELL - Use this to reset your table
conn = psycopg2.connect(**db_params)
conn.autocommit = True # Needed for DROP operations
cursor = conn.cursor()

# This will delete the table and all its data
cursor.execute("DROP TABLE IF EXISTS documents;")
print("Table 'documents' dropped.")

cursor.close()
conn.close()

Table 'documents' dropped.
